# Elera: reconhecimendo de cultivos irrigados
## Arquitetura e validação de modelo

Este notebook tem como objetivo o desenvolvimento e validação de uma rede neural artificial (ANN) capaz de identificar áreas de cultivo irrigadas e suas respectivas culturas, dentro do escopo de três categorias de interesse:
* "Cana-de-açúcar": Áreas de cultivo de cana-de-açúcar irrigadas.
* "Soja": Áreas de cultivo de soja irrigada, ou principalmente de outros cultivos irrigados localizados nas mesmas áreas que soja, mas em diferente sazonalidade. Exemplos comuns: milho e algodão.
* "Outras culturas perenes": Categoria que abrange diversos cultivos irrigados, especialmente frutíferas.

Parte dos dados aqui utilizados já foram pré-processados em etapas anteriores. Acesso aos scripts utilizados no processo estão disponíveis no repositório do projeto.

Este notebook utiliza serviços da Google que demandam credenciamento, tal como Google Cloud e Google Earth Engine. O usuário precisará de credenciais próprias para executar certas rotinas aqui presentes.

### Importação de bibliotecas

Declaração de módulos externos que serão utilizados no notebook.

In [1]:
#!pip install earthengine-api
#!pip install folium
#!pip install tensorflow
#!pip install geemap
import ee
import tensorflow as tf
#import geemap
from google.cloud import storage
import os
import json
from tensorflow.python.tools import saved_model_utils
from tensorflow import keras
import folium
import pandas as pd

### Autenticação 

Ativação da API por meio de credenciais do usuário.

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "pd-psr-elera-e8562bed1814.json"
service_account = 'rodrigobenoliel@pd-psr-elera.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'pd-psr-elera-e8562bed1814.json')
ee.Initialize(credentials)
storage_client = storage.Client.from_service_account_json('pd-psr-elera-e8562bed1814.json')

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
table = ee.FeatureCollection("projects/pd-psr-elera/assets/PNRH_SUB1_1")
image_irrigated = ee.Image('projects/pd-psr-elera/assets/irrigadas_2020')
studyArea = ee.FeatureCollection(table).geometry()
START_DATE = '2020-07-01'
END_DATE = '2020-07-30'


def cloud_shadow_Mask(image):
  qa = image.select('QA60')
  cloud = qa.bitwiseAnd(1<<10).Or(qa.bitwiseAnd(1<<11))

  mask2 = image.mask().reduce(ee.Reducer.min())
  return image.updateMask(cloud.Not()).updateMask(mask2)

image = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(studyArea).filterDate(START_DATE, END_DATE).map(cloud_shadow_Mask).median()

image = image.reproject(
      crs = image_irrigated.projection()
    ).reduceResolution(
      reducer = ee.Reducer.mean(),
      maxPixels = 1024
    )

image = ee.Image('projects/pd-psr-elera/assets/sent2_30m_2020-07-01_2020-07-30')

In [4]:
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      show=show,
      opacity=opacity,
      min_zoom=min_zoom,
      overlay=True,
      control=True
      ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

m = folium.Map(zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(image,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

### Leitura e processamento dos dados de entrada para o modelo

Inclui acesso direto a assets do projeto no Google Earth Engine. São estes:

* As áreas irrigadas de cultivo indefinido.
* Imagens de satélite Landsat 8.
* Fronteira que delimita a região de interesse do projeto.

Seleciona-se bandas específicas das imagens de satélite, que incluem cores no espectro visível e infravermelho. Em seguida, sintetiza-se uma única imagem com base em uma janela temporal de 15 dias, período compatível com os dados de irrigação.

In [5]:
#image = ee.Image("projects/pd-psr-elera/assets/Sentinel_2_cloud_free_2020-07-01_2020_07_15")
mapbiomas3 = ee.Image("projects/pd-psr-elera/assets/mapbiomas-brazil-collection-60-2020-0000000000-0000000000")

BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11','B12']
image = image.select(BANDS)
LABEL = 'class'
N_LABELS = 5
SCALE = 30

### Leitura de rótulos de referência a serem utilizados para treinamento do modelo

Os seguintes dados consistem em polígonos rotulados nas categorias de interesse que foram previamente selecionados selecionados. Serão definidas duas coleções:
* `data`: dados a serem usados para treinamento.
* `data_test_simple`: dados a serem usados para validação.
* `data_test`: banco de dados consideravelmente maior, que será usado para teste de desempenho do modelo.

In [6]:
TILE_SCALE = 16
data = ee.FeatureCollection('projects/pd-psr-elera/assets/uni_train_data_3')
data = image.sampleRegions(
    collection = data,
    properties = [LABEL],
    tileScale = TILE_SCALE,
    #scale = SCALE
    )

data_test_simple = ee.FeatureCollection('projects/pd-psr-elera/assets/uni_val_data_3')
data_test_simple = image.sampleRegions(
    collection = data_test_simple,
    properties = [LABEL],
    tileScale = TILE_SCALE,
    #scale = SCALE
    )

data_test_jp = ee.FeatureCollection('projects/pd-psr-elera/assets/uni_test_data_3_jp')
data_test_jp = image.sampleRegions(
    collection = data_test_jp,
    properties = [LABEL],
    tileScale = TILE_SCALE,
    #scale = SCALE
    )

data_test_jb = ee.FeatureCollection('projects/pd-psr-elera/assets/uni_test_data_3_jb')
data_test_jb = image.sampleRegions(
    collection = data_test_jb,
    properties = [LABEL],
    tileScale = TILE_SCALE,
    #scale = SCALE
    )



### Exportação de dados para o Google Cloud

Dados serão exportados para o Google Cloud em formato `TFRecord`. Este processo facilita o processo de conversão das estruturas de dados do Earth Engine para uma adequada ao framework TensorFlow, que será utilizada para o desenvolvimento do modelo.

As seguintes rotinas demandam credenciais próprias do Google Cloud para execução.


In [7]:
# Your Earth Engine username.  This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'rodrigobenoliel'

FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# Cloud Storage bucket into which training, testing and prediction 
# datasets will be written.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'elera_classificator'

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'uni_data_train_3'
TEST_SIMPLE_FILE_PREFIX = 'uni_data_val_3'
TEST_FILE_PREFIX_JP = 'uni_data_test_3_jp'
TEST_FILE_PREFIX_JB = 'uni_data_test_3_jb'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_SIMPLE_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_SIMPLE_FILE_PREFIX + file_extension
TEST_FILE_PATH_JP = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX_JP + file_extension
TEST_FILE_PATH_JB = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX_JB + file_extension



# Create the tasks.
training_task = ee.batch.Export.table.toCloudStorage(
  collection=data,
  description='Training Export',
  fileNamePrefix=TRAIN_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_simple_task = ee.batch.Export.table.toCloudStorage(
  collection=data_test_simple,
  description='Validation Export',
  fileNamePrefix=TEST_SIMPLE_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task_jp = ee.batch.Export.table.toCloudStorage(
  collection=data_test_jp,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX_JP,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task_jb = ee.batch.Export.table.toCloudStorage(
  collection=data_test_jb,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX_JB,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

print(FEATURE_NAMES)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'class']


Execução da exportação, este processo pode levar alguns segundos.

In [8]:
# Start the tasks.
testing_simple_task.start()
training_task.start()
testing_task_jp.start()
testing_task_jb.start()

In [9]:
import time

while testing_task_jb.active():
  print('Polling for task (id: {}).'.format(testing_task_jb.id))
  time.sleep(5)
print('Done with image export.')

Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id: DFISUW5DDXLLTVUA6UMNPOBW).
Polling for task (id

### Leitura de dados em formato TFRecord

Releitura dos dados no formato mais conveniente.

In [10]:
# Create a dataset from the TFRecord file in Cloud Storage.
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')
# Print the first record to check.
print(iter(train_dataset).next())
print(type(train_dataset))

tf.Tensor(b'\n\xb6\x01\n\x0e\n\x02B2\x12\x08\x12\x06\n\x04\x00\x80\xd0C\n\x0e\n\x02B3\x12\x08\x12\x06\n\x04\x00\x80$D\n\x0e\n\x02B4\x12\x08\x12\x06\n\x04\x00\xc0?D\n\x0e\n\x02B5\x12\x08\x12\x06\n\x04\x00\xa0\x90D\n\x0e\n\x02B6\x12\x08\x12\x06\n\x04\x00\xa0\xe6D\n\x0e\n\x02B7\x12\x08\x12\x06\n\x04\x00\x90\x04E\n\x0e\n\x02B8\x12\x08\x12\x06\n\x04\x00 \x07E\n\x0f\n\x03B8A\x12\x08\x12\x06\n\x04\x00\xf0\x12E\n\x0f\n\x03B11\x12\x08\x12\x06\n\x04\x00\x90"E\n\x0f\n\x03B12\x12\x08\x12\x06\n\x04\x00\xe0\xcdD\n\x11\n\x05class\x12\x08\x12\x06\n\x04\x00\x00\x00\x00', shape=(), dtype=string)
<class 'tensorflow.python.data.ops.readers.TFRecordDatasetV2'>


Definição de metadados dos parâmetros.

In [11]:
# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(FEATURE_NAMES, columns))

print(features_dict)

{'B2': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'B3': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'B4': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'B5': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'B6': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'B7': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'B8': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'B8A': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'B11': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'B12': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'class': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None)}


Definição de funções úteis para formatação dos dados e preparação para seu consumo pelo modelo. Funções são aplicadas aos dados de treinamento.

In [12]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
print(iter(parsed_dataset).next())

({'B11': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2601.], dtype=float32)>, 'B12': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1647.], dtype=float32)>, 'B2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([417.], dtype=float32)>, 'B3': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([658.], dtype=float32)>, 'B4': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([767.], dtype=float32)>, 'B5': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1157.], dtype=float32)>, 'B6': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1845.], dtype=float32)>, 'B7': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2121.], dtype=float32)>, 'B8': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2162.], dtype=float32)>, 'B8A': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2351.], dtype=float32)>}, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>)


In [13]:
# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def to_tuple(inputs, label):
  values = []
  for band in BANDS:
    values.append(inputs[band])
  return (tf.expand_dims(tf.transpose(values), 1),
          tf.expand_dims(tf.one_hot(indices=label, depth=N_LABELS),1))

# Map the to_tuple function, shuffle and batch.
input_dataset_unbatched = parsed_dataset.map(to_tuple)

print(iter(input_dataset_unbatched).next())
batch_size = 128
data_size = 395654
print("n rows", data_size)
input_dataset = input_dataset_unbatched.shuffle(140223).batch(batch_size)
print(type(input_dataset_unbatched))

(<tf.Tensor: shape=(1, 1, 10), dtype=float32, numpy=
array([[[ 417.,  658.,  767., 1157., 1845., 2121., 2162., 2351., 2601.,
         1647.]]], dtype=float32)>, <tf.Tensor: shape=(1, 1, 5), dtype=float32, numpy=array([[[1., 0., 0., 0., 0.]]], dtype=float32)>)
n rows 395654
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


### Definição do modelo

Definição da arquitetura da ANN e inicialização do seu treinamento e validação. 

In [16]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Input((None, None, len(BANDS))),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    #tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Dense(128, activation=tf.nn.relu),
    ##tf.keras.layers.Dense(128, activation=tf.nn.relu),
    #tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(N_LABELS, activation=tf.nn.softmax)
    ])

test_simple_dataset = (
  tf.data.TFRecordDataset(TEST_SIMPLE_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(to_tuple)
    .shuffle(140223)
    .batch(32))

opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

#checkpoint_filepath = '/tmp/checkpoint' 
#import datetime
#log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#%rm -rf "logs"
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_filepath, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True)
print(model.summary())
#%reload_ext tensorboard
model.fit(x=input_dataset, epochs=10, validation_data = test_simple_dataset)#, steps_per_epoch=data_size//batch_size)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_1 (Batc  (None, None, None, 10)   40        
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, None, None, 128)   1408      
                                                                 
 dropout_2 (Dropout)         (None, None, None, 128)   0         
                                                                 
 dense_5 (Dense)             (None, None, None, 128)   16512     
                                                                 
 dense_6 (Dense)             (None, None, None, 128)   16512     
                                                                 
 dropout_3 (Dropout)         (None, None, None, 128)   0         
                                                      

### Exportação do modelo para o Google Cloud

In [ ]:
MODEL_DIR = 'gs://' + OUTPUT_BUCKET + '/uni_model_3'
model.save(MODEL_DIR, save_format='tf')

INFO:tensorflow:Assets written to: gs://elera_classificator/uni_model_3/assets


In [15]:
MODEL_DIR = 'gs://' + OUTPUT_BUCKET + '/uni_model_3'
model = keras.models.load_model(MODEL_DIR)

from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

UnimplementedError: File system scheme 'gs' not implemented (file: 'gs://elera_classificator/uni_model_3')

### Testando modelo com banco de dados de teste

Esta etapa tem como objetivo testar um desempenho do modelo sob uma demanda mais estressante.

In [ ]:
MODEL_DIR = 'gs://' + OUTPUT_BUCKET + '/uni_model_3'
model = keras.models.load_model(MODEL_DIR)
data_in = tf.data.TFRecordDataset(TEST_FILE_PATH_JP, compression_type='GZIP').map(parse_tfrecord, num_parallel_calls=5)
print(iter(data_in).next())
eval_data = (data_in
    .map(to_tuple)
    .shuffle(140223)
    .batch(128))
print(eval_data)
model.evaluate(eval_data)

({'B11': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([3490.5], dtype=float32)>, 'B12': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2202.], dtype=float32)>, 'B2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([602.], dtype=float32)>, 'B3': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([870.5], dtype=float32)>, 'B4': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1182.5], dtype=float32)>, 'B5': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1575.], dtype=float32)>, 'B6': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2101.5], dtype=float32)>, 'B7': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2376.], dtype=float32)>, 'B8': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2573.], dtype=float32)>, 'B8A': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2768.], dtype=float32)>}, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>)
<BatchDataset element_spec=(TensorSpec(shape=(None, 1, 1, 10), dtype=tf.float32, name=None), 

[0.454293817281723, 0.8512747883796692]

In [ ]:
MODEL_DIR = 'gs://' + OUTPUT_BUCKET + '/uni_model_3'
model = keras.models.load_model(MODEL_DIR)
data_in = tf.data.TFRecordDataset(TEST_FILE_PATH_JB, compression_type='GZIP').map(parse_tfrecord, num_parallel_calls=5)
print(iter(data_in).next())
eval_data = (data_in
    .map(to_tuple)
    .shuffle(140223)
    .batch(128))
print(eval_data)
model.evaluate(eval_data)

({'B11': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2670.5], dtype=float32)>, 'B12': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1511.5], dtype=float32)>, 'B2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([285.], dtype=float32)>, 'B3': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([485.], dtype=float32)>, 'B4': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([468.5], dtype=float32)>, 'B5': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([914.5], dtype=float32)>, 'B6': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1796.5], dtype=float32)>, 'B7': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2178.], dtype=float32)>, 'B8': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2389.5], dtype=float32)>, 'B8A': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2623.5], dtype=float32)>}, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>)
<BatchDataset element_spec=(TensorSpec(shape=(None, 1, 1, 10), dtype=tf.float32, name=None),

[0.4708397686481476, 0.859380304813385]

In [ ]:
import seaborn as sns
import numpy as np
MODEL_DIR = 'gs://' + OUTPUT_BUCKET + '/uni_model_2'
model = keras.models.load_model(MODEL_DIR)
y_pred = model.predict(eval_data)
y_true = [value[1].numpy()[0][0][0] for value in eval_data]

KeyboardInterrupt: ignored

In [ ]:
y_pred = [np.argmax(pred) for pred in y_pred]
con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
sns.heatmap(con_mat, annot=True)

### EEfication do modelo

Este processo consiste na adaptação do modelo para que este possa ser exportado para a Google AI Platform, e em seguida possa ser utilizado diretamente por métodos do Google Earth Engine.

In [ ]:
OUTPUT_BUCKET = 'elera_classificator'
MODEL_DIR = 'gs://' + OUTPUT_BUCKET + '/uni_model_3'
model = keras.models.load_model(MODEL_DIR)
SERV_ACC = "rodrigobenoliel@pd-psr-elera.iam.gserviceaccount.com"

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs
# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break
print(inputs)


output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "class"}) + "'"
print(input_dict)
print(output_dict)

{'input_6': name: "serving_default_input_6:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
  dim {
    size: -1
  }
  dim {
    size: -1
  }
  dim {
    size: 10
  }
}
}
'{"serving_default_input_6:0": "array"}'
'{"StatefulPartitionedCall:0": "class"}'


In [ ]:
# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://' + OUTPUT_BUCKET + '/eeified_uni_model_3'
PROJECT = 'pd-psr-elera'
SERVICE_ACC_JSON = 'pd-psr-elera-e8562bed1814.json'
# You need to set the project before using the model prepare command.
#!earthengine authenticate
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Successfully saved project id
Success: model at 'gs://elera_classificator/eeified_uni_model_3' is ready to be hosted in AI Platform.


In [ ]:
REGION = 'us-central1'

MODEL_NAME = 'uni_classifier_3'
VERSION_NAME = 'v0'

#from google.cloud import storage
#storage_client = storage.Client.from_service_account_json(SERVICE_ACC_JSON)

! gcloud auth login #activate-service-account {SERV_ACC} --key-file={SERVICE_ACC_JSON}
! gcloud config set project {PROJECT}
!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT} \
  --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --region {REGION} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --framework "TENSORFLOW" \
  --runtime-version=2.3 \
  --python-version=3.7

You are authorizing gcloud CLI without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=hGD4I71W616fRKLFnE2e8bW7i5utnm&access_type=offline&code_challenge=68JnX51V23Tu8qPwA_y6D5S8t-cfzllSTqLdo4AcMFM&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=UnmnLjso15NrHQfkTr9EvoOgdUAdS1&code=4/0AX4XfWjFFe9reF3zD8l8rpQk2lP15EwePEOhi3MnX5IholmsZkQCRCuDaZPDunBmv

In [ ]:


AOI = studyArea
START_DATE = '2020-07-01'
END_DATE = '2020-10-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
  # Get s2cloudless image, subset the probability band.
  cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

  # Condition s2cloudless by the probability threshold value.
  is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

  # Add the cloud probability layer and cloud mask as image bands.
  return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      show=show,
      opacity=opacity,
      min_zoom=min_zoom,
      overlay=True,
      control=True
      ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer